In [ ]:
import os
import warnings
from os import getenv

import cv2
import nopdb
import numpy as np
import PIL
import torch
from matplotlib import pyplot as plt

from spoof.dataset.dataset import FaceDataset
from spoof.model.vit import ViT

warnings.filterwarnings("ignore")

In [ ]:
os.chdir(getenv("HOME") + "/mipt/spoof")

In [ ]:
# Define some functions to map the tensor back to an image
def inv_normalize(tensor):
    """Normalize an image tensor back to the 0-255 range."""
    tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min()) * (256 - 1e-5)
    return tensor


def inv_transform(tensor, normalize=True):
    """Convert a tensor to an image."""
    if normalize:
        tensor = inv_normalize(tensor)
    array = tensor.detach().cpu().numpy()
    array = array.transpose(1, 2, 0).astype(np.uint8)
    return PIL.Image.fromarray(array)

#### Tweaking the vision transformer source code

---
Here I load the model from our `ViT` class. To be able to capture the attention weights following modifications are made to the vision transformer source code:

- **`torchvision.models.vision_transformer.py`**, line `113`:

```
    class EncoderBlock(nn.Module):
    ...
    def forward(self, input: torch.Tensor):
        ...
        x, _ = self.self_attention(x, x, x, need_weights=True) # We need attention weights
        ...
```

- **`torch.nn.modules.activation.py`**, line `1026`:

```
    class MultiheadAttention(nn.Module):
    ...
    def forward(
            ...
            average_attn_weights: bool = False, # Easier if set to False
            ...
        ):
```
---

In [ ]:
# Create model and let it load the weights in the constructor
model = ViT()

#### Capture attention weights using ```nopdb``` library

In [ ]:
def capture_attn_call(input, layer_idx):
    with nopdb.capture_call(
        model.extractor.encoder.layers[layer_idx].self_attention.forward
    ) as attn_call:
        with torch.no_grad():
            model(input)
    return attn_call


# Get the attention matrix for the last layer for an image returned from from the dataset iterator
img = next(iter(FaceDataset("data/casia/train/annotations.csv")))["image"]
attn_matrix = capture_attn_call(img, 11).locals["attn_output_weights"][0]
print(f"Attention matrix shape: {attn_matrix.shape}")

#### Visualize attention map 

- I modified this function from the kaggle link you sent me, I'm not 100 % clear what's going on here but it worked after modifying a bit.

- Other visualization implementations didn't work quite well for me.

In [ ]:
def get_attention_map(img, layer_idx, get_mask=False):
    """Get the attention map for an image."""
    attn_matrix = capture_attn_call(img, layer_idx).locals["attn_output_weights"]

    # Average the attention weights across all heads.
    attn_matrix = torch.mean(attn_matrix, dim=1)

    # To account for residual connections, we add an identity matrix to the
    # attention matrix and re-normalize the weights.
    residual_att = torch.eye(attn_matrix.size(1))
    aug_att_mat = attn_matrix + residual_att
    aug_att_mat = aug_att_mat / aug_att_mat.sum(dim=-1).unsqueeze(-1)

    # Recursively multiply the weight matrices
    joint_attentions = torch.zeros(aug_att_mat.size())
    joint_attentions[0] = aug_att_mat[0]

    for n in range(1, aug_att_mat.size(0)):
        joint_attentions[n] = torch.matmul(aug_att_mat[n], joint_attentions[n - 1])

    v = joint_attentions[-1]
    grid_size = int(np.sqrt(aug_att_mat.size(-1)))
    mask = v[0, 1:].reshape(grid_size, grid_size).detach().numpy()

    if get_mask:
        result = cv2.resize(mask / mask.max(), (img.shape[1], img.shape[2]))
    else:
        mask = cv2.resize(mask / mask.max(), (img.shape[1], img.shape[2]))[..., np.newaxis]
        result = mask * img.numpy().transpose(1, 2, 0)

    return result

In [ ]:
def plot_attention(img, save_path=None):
    """Plot the attention maps for each layer."""
    # Get the attention maps for each layer with and without mask and combine them into a single list
    attns_nomask = [get_attention_map(img, i) for i in range(12)]
    attns_mask = [get_attention_map(img, i, get_mask=True) for i in range(12)]
    attns = [None] * (len(attns_nomask) + len(attns_mask))
    attns[::2] = attns_nomask
    attns[1::2] = attns_mask
    fn = os.path.basename(save_path)
    # Plot the attention maps for each layer with and without mask side by side
    fig = plt.figure(figsize=(32, 32))
    for i, attn in enumerate(attns):
        ax = fig.add_subplot(4, 6, i + 1)
        ax.imshow(attn)
        ax.axis("off")
        if i % 2 == 0:
            ax.set_title(f"Layer {i // 2} (w/o mask)", fontsize=24)
        else:
            ax.set_title(f"Layer {i // 2} (w/ mask)", fontsize=24)
        fig.tight_layout()
        fig.suptitle(f"Attention Maps for {fn}", fontsize=32)

    # Save
    if save_path is not None:
        plt.savefig(save_path, bbox_inches="tight")
        plt.close("all")
    else:
        fig.show()

#### Visualize attention map

In [ ]:
plot_attention(img, save_path="attn_map.png")

#### Save a couple of attention maps for Casia & Replay train and test splits

Here I preserved the folder structure of what I have locally for both datasets and saved a number of above figures for you to examine.

In [ ]:
num_samples = 10
split = ["train", "test"]

# create subfolders for each split, sorry if paths are not Windows compatible, I will fix it later
os.makedirs("figures/attention/casia/train", exist_ok=True)
os.makedirs("figures/attention/casia/test", exist_ok=True)

for s in split:
    casia = FaceDataset(f"data/casia/{s}/annotations.csv")
    for i in range(num_samples):
        idx = np.random.randint(len(casia))
        img = casia[idx]["image"]
        fn = casia[idx]["filename"].split("/")[-1]
        folder = f"figures/attention/casia/{s}"
        save_path = f"{folder}/{fn}"
        plot_attention(img, save_path=save_path)

#### Replay

In [ ]:
# kind = ["real", "attack"]

# # create subfolders for each split
# for k in kind:
#     os.makedirs(f"figures/attention/replay/train/{k}", exist_ok=True)
#     os.makedirs(f"figures/attention/replay/test/{k}", exist_ok=True)

# for s in split:
#     replay = FaceDataset(f"data/replay/{s}/annotations.csv")
#     for k in kind:
#         for i in range(num_samples):
#             idx = np.random.randint(len(replay))
#             img = replay[idx]["image"]
#             fn = replay[idx]["filename"].split("/")[-1]
#             save_path = f"figures/attention/replay/{s}/{k}/{fn}"
#             plot_attention(img, save_path=save_path)